In [16]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from astropy.io import ascii
from utils import col_names, relevant_indices, normalise_sdss_class

In [3]:
datasets = "../datasets/SuperCOSMOS/"

In [4]:
%%time
uki823_df = ascii.read(datasets + "UKI823/sssedrpair.dat").to_pandas()
ukr823_df = ascii.read(datasets + "UKR823/sssedrpair.dat").to_pandas()
ukj823_df = ascii.read(datasets + "UKJ823/sssedrpair.dat").to_pandas()

CPU times: user 3.86 s, sys: 151 ms, total: 4.01 s
Wall time: 4.01 s


In [5]:
uki823_df.columns = col_names
ukr823_df.columns = col_names
ukj823_df.columns = col_names

In [20]:
uki823_df[['CLASS', 'CLASS_SDSS']]
confusion_matrix(uki823_df['CLASS'], uki823_df['CLASS_SDSS'])

array([[4537, 3612,    0,    0],
       [1225, 6191,    0,    0],
       [  11,    2,    0,    0],
       [  40,   27,    0,    0]])